File for additional testing to verify https://redmine.deriv.cloud/issues/96450#%22sell%22_API_call_sometimes_doesn't_sell_at_market

In [1]:
import requests
import websocket
import json
from websocket import create_connection
import time
import pandas as pd
import numpy as np
from scipy.stats import norm
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_colwidth',300)
from IPython.display import clear_output
import random
from scipy.optimize import fsolve

# from VanillasFunctions import add,barrier_check

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [116]:
# Volatility index parameters such as vol, displayed decimals
assets=pd.read_csv("assets.csv")
#API URL
master_URL = "wss://qa142.deriv.dev/websockets/v3?app_id=1007&l=EN&brand=deriv"
#api token
api_token = "wmRauguRX8I2Jdn"

In [100]:
def barrier_check (symbol, duration, duration_unit, contract_type): 
    return_json = []
    def on_open(ws):
        msg = {
            "authorize": api_token,
            "req_id": 1
        }
        ws.send(json.dumps(msg))
    def on_message(ws, message):
        message = json.loads(message)
        print(message)
        if 'authorize' in message:
            if duration_unit == "d":
                barrier = "10000"
            else:
                barrier = "+0"
            msg = {
                "proposal": 1,
                "amount": 60,
                "barrier": barrier,
                "basis": "stake",
                "contract_type": contract_type,
                "currency": "AUD",
                "duration": duration,
                "duration_unit": duration_unit,
                "symbol": symbol
            }
            print('sendbar')
            ws.send(json.dumps(msg))
        else:
            print('here')
            amount = 60
            if 'error' in message.keys():
                barrier = message['error']['details']['barrier_choices'][int(len(message['error']['details']['barrier_choices'])/2)]
                if message['error']['details']['field'] == 'amount':
                    amount = float(message['error']['details']['max_stake'])
                    print('amount =',amount)
                    print(barrier)
                    print(contract_type)
                    print(duration)
                    print(duration_unit)
                    print(symbol)
                msg = {
                    "proposal": 1,
                    "amount": amount,
                    "barrier": barrier,
                    "basis": "stake",
                    "contract_type": contract_type,
                    "currency": "AUD",
                    "duration": duration,
                    "duration_unit": duration_unit,
                    "symbol": symbol
                }
                print('sent')
                ws.send(json.dumps(msg))   
            else:
                return_json.append(message)
                ws.close()
    def on_close(ws, close_status_code, close_msg):
        print()             
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    barriers = return_json[-1]['proposal']['barrier_choices']
    return return_json[-1]

In [101]:
def calc_barriers (pd_series): #(spot, symbol, duration, duration_unit):   
    spot = float(pd_series.Spot)    
    symbol = pd_series.Asset    
    duration = pd_series.Start_expiry    
    duration_unit = pd_series.Duration[-1]    
    digits = int( assets[assets['Asset']== symbol].digits )    
    sigma = float( assets[assets['Asset']==symbol].IV )      
    n_digits = round( - np.log10(spot * sigma * np.sqrt(60/365/86400)) , 0) - 1    
    spot = round(spot/10**-n_digits)*10**-n_digits    
    t = (int(duration[1]) - int(duration[0])) / 365 / 86400           
    # Uncomment this section to match all API barriers to self calculated barriers
    if t < 5 * 60 / 365 / 86400 and 'R' in symbol:
        t -= 2 / 365 / 86400       
    if duration_unit != 'd':        
        deltas = np.arange(1,10,2)/10     
        real_strikes = spot * np.exp( (sigma**2)/2*t - (sigma * np.sqrt(t)  * norm.ppf(deltas)) )    
        displayed_barriers = np.where( deltas > 0.5, np.round((real_strikes - spot)*10**digits)/10**digits , np.round((real_strikes - spot)*10**digits)/10**digits )        
    else:        
        temp_barriers = []        
        displayed_barriers = []
        central_strike = round( spot / (10 ** -n_digits) ) * (10 ** - n_digits )
        max_strike =  np.floor( (spot * np.exp((sigma**2)/2*t - (sigma * np.sqrt(t)  * norm.ppf(0.1)))) / (10 ** -n_digits ) ) * (10 ** -n_digits )
        min_strike =  np.ceil( (spot * np.exp((sigma**2)/2*t - (sigma * np.sqrt(t)  * norm.ppf(0.9)))) / (10 ** -n_digits ) ) * (10 ** -n_digits )
        strike_step_1 = round( (central_strike - min_strike) / 5 / 10 ) * 10
        strike_step_2 = round( (max_strike - central_strike) / 5 / 10 ) * 10 
        for i in range(5):
            temp_barriers.append(min_strike + strike_step_1*i)
        for i in range(5):
            temp_barriers.append(central_strike + strike_step_2*i)
        temp_barriers.append(max_strike)
        for strike in temp_barriers:
            if strike not in displayed_barriers:
                displayed_barriers.append(strike)
    return displayed_barriers       

In [102]:
def calc_ppp_ask (pd_series, t_step, s_step):
    symbol = pd_series.Asset    
    duration = pd_series.Start_expiry    
    duration_unit = pd_series.Duration[-1]    
    stake = pd_series.Stake
    vol_charge = 1.025 
    t = ( int(duration[1]) - int(duration[0]) + t_step ) / 365 / 86400      
    if t < 5 * 60 / 365 / 86400 and 'R' in symbol:
        print('rounded')
        t -= 2 / 365 / 86400
    r = pd_series.Discount_rate     
    contract_type = pd_series.Contract_type
    number_of_contracts = pd_series.API_PPP        
    if s_step == 0:        
        spot = float(pd_series.Spot)
        strike = pd_series.Barrier    
    else:        
        spot = float(tick_history(int(duration[0]), symbol, s_step))        
        if duration_unit != 'd':            
            strike = spot + float(pd_series.Displayed_barrier)            
        else:            
            strike = pd_series.Barrier        
    sigma = float( assets[assets['Asset']==symbol].IV )    
    if duration_unit != 'd':        
        spot_spread = float( assets[assets['Asset']==symbol].SS )    
        bs_markup = float( assets[assets['Asset']==symbol].BS )        
    else:        
        spot_spread = float( assets[assets['Asset']==symbol].SSD )    
        bs_markup = float( assets[assets['Asset']==symbol].BSD )        
    vol = sigma * vol_charge
    d1 = (np.log(spot/strike) + (vol**2/2)*t)/(vol*np.sqrt(t))
    d2 = (d1 - vol*np.sqrt(t))    
    if contract_type == "VANILLALONGCALL":        
        phi = 1        
        delta_charge = (norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t)))) * spot_spread / 2        
    elif  contract_type == "VANILLALONGPUT":        
        phi = -1       
        delta_charge = (abs(norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t))) - 1)) * spot_spread / 2        
    bs_price = (phi*np.exp(-r*t)*(spot*norm.cdf(phi*d1) - strike*norm.cdf(phi*d2)))    
    print(symbol,'s', spot, 'k',strike, r, t, vol,  phi,  'p',number_of_contracts, stake, 'd', (norm.cdf(phi*d1)))    
    return stake / ( bs_price + delta_charge + bs_markup )

In [103]:
def tick_history (tick,symbol,t):
    
    if 'R' in symbol:
        r = 2
    else:
        r = 1
        
    temp_json = []
    
    def on_open(ws):
        omsg = {
          "ticks_history": symbol,
          "adjust_start_time": 1,
          "count": 5,
          "end": tick+t*r,
          "start": 1,
          "style": "ticks"
        }
        ws.send(json.dumps(omsg))
        
    def on_message(ws, message):

        message = json.loads(message)
        print('\n',message)
        if 'authorize' in message:
            ws.send(json.dumps(msg))
            
        else:        
            temp_json.append(message)
            ws.close()
    
    def on_close(ws, close_status_code, close_msg):
        print()
        
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    
    return temp_json[-1]['history']['prices'][-1]
    

In [104]:
def get_proposal (contract_id): 
    
    contract_id = int(contract_id)
    
    return_json = []
    
    def on_open(ws):
        
        msg = {
            "authorize": api_token,
            "req_id": 1
        }
        
        ws.send(json.dumps(msg))


    def on_message(ws, message):

        message = json.loads(message)
        print(message)
        if 'authorize' in message:
            
            msg = {
              "proposal_open_contract": 1,
              "contract_id": contract_id
            }
            
            ws.send(json.dumps(msg))
            
        else:
                    
            return_json.append(message)
            
            ws.close()

    def on_close(ws, close_status_code, close_msg):
        print()

    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    
    return return_json[-1]

In [105]:
def get_barriers (symbol, duration, duration_unit, contract_type): 
    barrier = 0
    if duration_unit == "d":
        barrier = "10000"
    else:
        barrier = "+0"
    return_json = []
    def on_open(ws):
        msg = {
            "authorize": api_token,
            "req_id": 1
        }
        ws.send(json.dumps(msg))
    def on_message(ws, message):
        message = json.loads(message)
        if duration_unit == "d":
            barrier = "10000"
        else:
            barrier = "+0" 
        print('\n',message)
        if 'authorize' in message:
            
            msg = {
                "proposal": 1,
                "amount": 60,
                "barrier": barrier,
                "basis": "stake",
                "contract_type": contract_type,
                "currency": "AUD",
                "duration": duration,
                "duration_unit": duration_unit,
                "symbol": symbol
            }
            ws.send(json.dumps(msg))
        else:
            amount = 60
            if 'error' in message.keys():
                if message['error']['details']['field'] == 'barrier':
                    barrier = message['error']['details']['barrier_choices'][np.random.randint(len(message['error']['details']['barrier_choices']))]
                elif message['error']['details']['field'] == 'amount':
                    amount = int(message['error']['details']['min_stake'])
                    barrier = message['echo_req']['barrier']

                print(amount)
                print(barrier)
                print(contract_type)
                print(duration)
                print(duration_unit)
                print(symbol)
                msg = {
                    "proposal": 1,
                    "amount": amount,
                    "barrier": barrier,
                    "basis": "stake",
                    "contract_type": contract_type,
                    "currency": "AUD",
                    "duration": duration,
                    "duration_unit": duration_unit,
                    "symbol": symbol
                }
                print(msg)
                ws.send(json.dumps(msg))   

            else:

                return_json.append(message)
                ws.close()          
    def on_close(ws, close_status_code, close_msg):
        print()              
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()    
    barriers = return_json[-1]['proposal']['barrier_choices']    
    if 'qa' in master_URL:        
        return barriers, return_json[-1]['proposal']['pricing_args']['discount_rate']    
    else:        
        return barriers, 0

In [106]:
def get_contract (symbol, duration, duration_unit, contract_type): 
#     temp_unit = [duration_unit]
    return_json = []
    return_barrier = []   
    return_discount_rate = []
    contract_ID = []    
    def on_open(ws):        
        msg = {
            "authorize": api_token,
        }
        ws.send(json.dumps(msg))
    def on_message(ws, message):
        print('\n',message)
        barrier = 0        
        amount = 60        
        message = json.loads(message)        
#         print('\n',message)
#         print('here1')
        if 'authorize' in message:
            barriers = get_barriers(symbol, duration, duration_unit, contract_type)
            barrier = barriers[0][np.random.randint(len(barriers[0]))]
            return_barrier.append(barrier)
            return_discount_rate.append(barriers[1])
            msg = {
                "buy": "1",
                "parameters" :
                {
                    "amount": amount,
                    "barrier": barrier,
                    "basis": "stake",
                    "contract_type": contract_type,
                    "currency": "AUD",
                    "duration": duration,
                    "duration_unit": duration_unit,
                    "symbol": symbol
                },
                "price": amount
            }
            ws.send(json.dumps(msg))    
        elif 'error' in message.keys():
            if 'code' in message['error'].keys():
                amount = float(message['error']['message'].split()[-1][:-1])                
                if 'ax' in message['error']['message']:
                    amount = float(message['error']['message'].split()[-1][:-1])
                else: 
                    amount += 5                    
                print('here',amount)
            print('here2',amount)    
            barriers = get_barriers(symbol, duration, duration_unit, contract_type)
            barrier = barriers[0][np.random.randint(len(barriers[0]))]
            return_barrier.append(barrier)
            return_discount_rate.append(barriers[1])           
            print (symbol)            
            msg = {
                "buy": "1",
                "parameters" :
                {
                    "amount": amount,
                    "barrier": barrier,
                    "basis": "stake",
                    "contract_type": contract_type,
                    "currency": "AUD",
                    "duration": duration,
                    "duration_unit": duration_unit,
                    "symbol": symbol
                },
                "price": amount
            }
            print('msgcre')
            print(msg)
            ws.send(json.dumps(msg))
            print('sent')
        elif 'buy' in message:
            print('bought')
            contract_ID.append(message['buy']['contract_id'])

            msg = {
              "proposal_open_contract": 1,
              "contract_id": contract_ID[-1]
            }
            ws.send(json.dumps(msg))            
        else:
            return_json.append(message)            
            ws.close()                                          
    def on_close(ws, close_status_code, close_msg):
        print()
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()   
    return return_json[-1], return_discount_rate[-1], return_barrier[-1]

In [107]:
def calc_ppp_bid (proposal, t_step, s_step):
    symbol = proposal['proposal_open_contract']['underlying']    
    stake = float(proposal['proposal_open_contract']['bid_price'])
    vol_charge = 0.975 
    t = ( proposal['proposal_open_contract']['date_expiry'] - proposal['proposal_open_contract']['current_spot_time'] + t_step ) / 365 / 86400          
    if t < 5 * 60 / 365 / 86400 and 'R' in symbol:
        t -= 2 / 365 / 86400        
    if t > 1 / 365:        
        r = proposal['proposal_open_contract']['pricing_args']['discount_rate']        
        spot_spread = float( assets[assets['Asset']==symbol].SSD )    
        bs_markup = float( assets[assets['Asset']==symbol].BSD )        
    else:        
        r = 0                
        spot_spread = float( assets[assets['Asset']==symbol].SS )    
        bs_markup = float( assets[assets['Asset']==symbol].BS )    
    contract_type = proposal['proposal_open_contract']['contract_type']
    number_of_contracts = float(proposal['proposal_open_contract']['number_of_contracts'])                
    if s_step == 0:        
        spot = float(proposal['proposal_open_contract']['current_spot'])
        strike = float(proposal['proposal_open_contract']['barrier'])    
    else:
        spot = float(tick_history(int(proposal['proposal_open_contract']['current_spot_time']), symbol, s_step))        
        if t > 1 / 365:        
            strike = float(proposal['proposal_open_contract']['barrier'])
        else:
            strike = spot + float(proposal['proposal_open_contract']['barrier']) - float(proposal['proposal_open_contract']['current_spot'])    
    sigma = float( assets[assets['Asset']==symbol].IV )        
    if contract_type == "VANILLALONGCALL":        
        phi = 1        
        delta_charge = (norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t)))) * spot_spread / 2
    elif  contract_type == "VANILLALONGPUT":        
        phi = -1        
        delta_charge = (abs(norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t))) - 1)) * spot_spread / 2    
    print(spot,strike,r, t, phi,  number_of_contracts, stake)
    vol = sigma * vol_charge
    d1 = (np.log(spot/strike) + (vol**2/2)*t)/(vol*np.sqrt(t))
    d2 = (d1 - vol*np.sqrt(t))
    return ( (phi*np.exp(-r*t)*(spot*norm.cdf(phi*d1) - strike*norm.cdf(phi*d2))) - delta_charge - bs_markup) * number_of_contracts    

In [108]:
def sell_contract(contract_id):    
    contract_id = int(contract_id)    
    temp_json = []    
    def on_open(ws):
        msg = {
            "authorize": api_token,
            "req_id": 1
        }
        ws.send(json.dumps(msg))        
    def on_message(ws, message):
        message = json.loads(message)
        print('\n',message)
        if 'authorize' in message:            
            msg = {
              "sell": contract_id,
              "price": 0
            }
            ws.send(json.dumps(msg))           
        else:        
            temp_json.append(float(message['sell']['sold_for']))
            ws.close()    
    def on_close(ws, close_status_code, close_msg):
        print()        
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()    
    return temp_json[-1]

In [109]:
def calc_ppp_sold (proposal, t_step, s_step):
    symbol = proposal['proposal_open_contract']['underlying']    
    stake = float(proposal['proposal_open_contract']['sell_price'])
    vol_charge = 0.975 
    t = ( proposal['proposal_open_contract']['date_expiry'] - proposal['proposal_open_contract']['exit_tick_time'] + t_step) / 365 / 86400  
    if t > 1 / 365:        
        r = proposal['proposal_open_contract']['pricing_args']['discount_rate']        
        spot_spread = float( assets[assets['Asset']==symbol].SSD )    
        bs_markup = float( assets[assets['Asset']==symbol].BSD )        
    else:        
        r = 0                
        spot_spread = float( assets[assets['Asset']==symbol].SS )
        bs_markup = float( assets[assets['Asset']==symbol].BS )    
    contract_type = proposal['proposal_open_contract']['contract_type']
    number_of_contracts = float(proposal['proposal_open_contract']['number_of_contracts'])        
    if s_step == 0:        
        spot = float(proposal['proposal_open_contract']['exit_tick'])
        strike = float(proposal['proposal_open_contract']['barrier'])    
    else:
        spot = float(tick_history(proposal['proposal_open_contract']['exit_tick_time'], symbol, s_step))        
        strike = float(proposal['proposal_open_contract']['barrier'])    
    sigma = float( assets[assets['Asset']==symbol].IV )
    if contract_type == "VANILLALONGCALL":        
        phi = 1        
        delta_charge = (norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t)))) * spot_spread / 2
    elif  contract_type == "VANILLALONGPUT":        
        phi = -1
        delta_charge = (abs(norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t))) - 1)) * spot_spread / 2        
    print('s',spot,'k',strike,'r',r,'t', t, phi,'n',  number_of_contracts,'s', stake)
    vol = sigma * vol_charge
    d1 = (np.log(spot/strike) + (vol**2/2)*t)/(vol*np.sqrt(t))
    d2 = (d1 - vol*np.sqrt(t))    
    price = ( (phi*np.exp(-r*t)*(spot*norm.cdf(phi*d1) - strike*norm.cdf(phi*d2))) - delta_charge - bs_markup)
    print(price)
    return max(price * number_of_contracts    ,0)

In [110]:
def calc_ppp_exp (proposal, t_step, s_step):
    stake = float(proposal['proposal_open_contract']['sell_price'])
    contract_type = proposal['proposal_open_contract']['contract_type']
    number_of_contracts = float(proposal['proposal_open_contract']['number_of_contracts'])
    if s_step == 0:
        spot = float(proposal['proposal_open_contract']['exit_tick'])
        strike = float(proposal['proposal_open_contract']['barrier'])
    else:
        spot = float(tick_history(proposal['proposal_open_contract']['exit_tick_time'], symbol, s_step))        
#         strike = spot - float(proposal['proposal_open_contract']['exit_tick']) + float(proposal['proposal_open_contract']['barrier'])
        strike = float(proposal['proposal_open_contract']['barrier'])    
    if contract_type == "VANILLALONGCALL":
        phi = 1
    elif  contract_type == "VANILLALONGPUT":
        phi = -1
    print(spot,strike, phi,  number_of_contracts, stake)
    return max( (spot - strike) * phi * number_of_contracts, 0)

In [119]:
# Sold_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Stake', 'Duration', \
#                                'Displayed_barrier', 'Barrier', 'Start_expiry', 'ID', 'Discount_rate', 'API_PPP','json'])
contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]
time_units = ('m', 'h', 'd')
while True:
    print(len(Sold_df))
    if len(Sold_df) > 50 - 1:
        break
    contract_type = random.choice(contract_types)
    asset = random.choice(assets.Asset)
    asset = 'R_100'
    duration_unit = random.choice(['m','h','d'])
    if duration_unit == 'm':
        max_duration = 1440
    elif duration_unit == 'h':
        max_duration = 24
    elif duration_unit == 'd':
        max_duration = 365
    i= np.random.randint(6)
    duration = np.random.randint(max_duration/2, max_duration)
    contract = get_contract(asset, duration, duration_unit, contract_type)
    proposal = contract[0]
    discount_rate = contract[1]
    displayed_barrier = contract[2]
    Sold_df.loc[len(Sold_df)] = [proposal['proposal_open_contract']['underlying'], proposal['proposal_open_contract']['contract_type'], \
                               proposal['proposal_open_contract']['entry_tick'], proposal['proposal_open_contract']['entry_tick_time'], \
                               float(proposal['proposal_open_contract']['buy_price']), str(duration)+duration_unit, displayed_barrier, \
                               float(proposal['proposal_open_contract']['barrier']), [proposal['proposal_open_contract']['date_start'], \
                               proposal['proposal_open_contract']['date_expiry']], proposal['echo_req']['contract_id'], discount_rate, \
                               float(proposal['proposal_open_contract']['number_of_contracts']), proposal]

    clear_output(wait=True)    
Sold_df

50


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,json
0,R_100,VANILLALONGPUT,10258.54,1689146636,60.0,15h,+235.00,10493.54,"[1689146636, 1689200636]",14199,0.000000,0.18897,"{'echo_req': {'contract_id': 14199, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200461', 'barrier': '10493.54', 'barrier_count': 1, 'bid_price': 58.61, 'buy_price': '60.00', 'contract_id': '14199', 'contract_type': 'VANILLALONGPUT..."
1,R_100,VANILLALONGCALL,10267.17,1689146640,60.0,1281m,-251.00,10016.17,"[1689146640, 1689223500]",14219,0.000000,0.16939,"{'echo_req': {'contract_id': 14219, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200461', 'barrier': '10016.17', 'barrier_count': 1, 'bid_price': 58.51, 'buy_price': '60.00', 'contract_id': '14219', 'contract_type': 'VANILLALONGCAL..."
2,R_100,VANILLALONGCALL,10274.71,1689146644,60.0,210d,31100.00,31100.00,"[1689146644, 1707350399]",14239,0.047267,0.13167,"{'echo_req': {'contract_id': 14239, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200461', 'barrier': '31100.00', 'barrier_count': 1, 'bid_price': 46.87, 'buy_price': '60.00', 'contract_id': '14239', 'contract_type': 'VANILLALONGCAL..."
3,R_100,VANILLALONGCALL,10267.64,1689146646,60.0,1184m,-241.90,10025.74,"[1689146647, 1689217687]",14259,0.000000,0.17593,"{'echo_req': {'contract_id': 14259, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200461', 'barrier': '10025.74', 'barrier_count': 1, 'bid_price': 58.12, 'buy_price': '60.00', 'contract_id': '14259', 'contract_type': 'VANILLALONGCAL..."
4,R_100,VANILLALONGPUT,10266.21,1689146650,60.0,12h,+7.10,10273.31,"[1689146652, 1689189852]",14279,0.000000,0.37737,"{'echo_req': {'contract_id': 14279, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200461', 'barrier': '10273.31', 'barrier_count': 1, 'bid_price': 57.94, 'buy_price': '60.00', 'contract_id': '14279', 'contract_type': 'VANILLALONGPUT..."
5,R_100,VANILLALONGPUT,10253.06,1689146690,194.0,339d,7000.00,7000.00,"[1689146691, 1718495999]",14319,0.046496,0.10839,"{'echo_req': {'contract_id': 14319, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200461', 'barrier': '7000.00', 'barrier_count': 1, 'bid_price': 178.46, 'buy_price': '194.00', 'contract_id': '14319', 'contract_type': 'VANILLALONGPU..."
6,R_100,VANILLALONGCALL,10250.41,1689146694,60.0,1207m,+274.30,10524.71,"[1689146695, 1689219115]",14339,0.000000,0.60362,"{'echo_req': {'contract_id': 14339, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200461', 'barrier': '10524.71', 'barrier_count': 1, 'bid_price': 50.15, 'buy_price': '60.00', 'contract_id': '14339', 'contract_type': 'VANILLALONGCAL..."
7,R_100,VANILLALONGCALL,10260.61,1689146700,66.0,344d,10300.00,10300.00,"[1689146701, 1718927999]",14359,0.046466,0.01763,"{'echo_req': {'contract_id': 14359, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200461', 'barrier': '10300.00', 'barrier_count': 1, 'bid_price': 62.75, 'buy_price': '66.00', 'contract_id': '14359', 'contract_type': 'VANILLALONGCAL..."
8,R_100,VANILLALONGCALL,10265.31,1689146706,60.0,20h,+11.80,10277.11,"[1689146706, 1689218706]",14379,0.000000,0.30132,"{'echo_req': {'contract_id': 14379, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200461', 'barrier': '10277.11', 'barrier_count': 1, 'bid_price': 54.46, 'buy_price': '60.00', 'contract_id': '14379', 'contract_type': 'VANILLALONGCAL..."
9,R_100,VANILLALONGCALL,10252.30,1689146756,644.1,1119m,-581.10,9671.20,"[168

In [120]:
calc_sold = []
sold_time = []
api_sold = []
sold_prop = []
for i in Sold_df.index:    
    if i >= len(sold_time):       
        clear_output(wait = True)        
        sell_contract(Sold_df.loc[i].ID)        
        proposal = get_proposal(Sold_df.loc[i].ID)
        sold_time.append(proposal['proposal_open_contract']['exit_tick_time'])
        api_sold.append(float(proposal['proposal_open_contract']['sell_price']))
        calc_sold.append( calc_ppp_sold(proposal, 0, 0) )        
        sold_prop.append(proposal)    
Sold_df['Sold_time'] = sold_time
Sold_df['API_sold'] = api_sold
Sold_df['Calc_sold'] = calc_sold
Sold_df['Sold_prop'] = sold_prop
clear_output(wait = True)
print(len(Sold_df), 'Test cases','\nMismatches =',\
      len(Sold_df[(Sold_df.API_sold) - np.round(Sold_df.Calc_sold,2) != 0]),\
      '\nMismatches listed below :')
df = Sold_df[(Sold_df.API_sold) - np.round(Sold_df.Calc_sold,2) != 0]
df

50 Test cases 
Mismatches = 2 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,json,Sold_time,API_sold,Calc_sold,Sold_prop
44,R_100,VANILLALONGCALL,10234.39,1689146898,60.0,746m,+512.10,10746.49,"[1689146899, 1689191659]",15119,0.0,2.48182,"{'echo_req': {'contract_id': 15119, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200461', 'barrier': '10746.49', 'barrier_count': 1, 'bid_price': 30.26, 'buy_price': '60.00', 'contract_id': '15119', 'contract_type': 'VANILLALONGCAL...",1689147258,20.15,20.155443,"{'echo_req': {'contract_id': 15119, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200461', 'audit_details': {'contract_start': [{'epoch': 1689146894, 'tick': '10231.63', 'tick_display_value': '10231.63'}, {'epoch': 1689146896, 'tick..."
45,R_100,VANILLALONGPUT,10234.59,1689146902,60.0,17h,-223.30,10011.29,"[1689146903, 1689208103]",15139,0.0,0.62292,"{'echo_req': {'contract_id': 15139, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200461', 'barrier': '10011.29', 'barrier_count': 1, 'bid_price': 49.76, 'buy_price': '60.00', 'contract_id': '15139', 'contract_type': 'VANILLALONGPUT...",1689147262,57.01,57.015421,"{'echo_req': {'contract_id': 15139, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200461', 'audit_details': {'contract_start': [{'epoch': 1689146898, 'tick': '10234.39', 'tick_display_value': '10234.39'}, {'epoch': 1689146900, 'tick..."
